ジェンダーを軸にして、クロス集計する

In [1]:
import altair as alt
import pandas as pd
import scipy as sp
import titanite as ti

print(f"Altair {alt.__version__}")
print(f"Pandas {pd.__version__}")
print(f"SciPy {sp.__version__}")
print(f"Titanite {ti.__version__}")

Altair 5.0.1
Pandas 2.0.3
SciPy 1.11.1
Titanite 0.4.0


データを読み込む

In [2]:
f_cfg = "../sandbox/config.toml"
f_csv = "../data/test_data/prepared_data.csv"
d = ti.Data(read_from=f_csv, load_from=f_cfg)
data = d.read()
# data.info()

2023-07-23 12:12:32.615 | INFO     | titanite.preprocess:categorical_data:123 - Categorize
2023-07-23 12:12:32.644 | INFO     | titanite.preprocess:binned_data:229 - Binned


In [3]:
x = ["q02"]
headers = d.crosstab_headers(x, list(data.columns))

In [4]:
cross_tabs, heatmaps, chi2_data = ti.core.crosstab_loop(data, headers)
chi2_data

,questions,p_value,statistic,dof
0,q02-q01,0.579596,29.775320,32
1,q02-q03,0.243178,71.488371,64
2,q02-q04,0.192958,77.894940,68
3,q02-q05,0.632957,21.097674,24
4,q02-q06,0.000047,73.110661,32
5,q02-q07,0.769531,4.887485,8
6,q02-q08,0.000002,63.665246,20
7,q02-q09,0.303268,9.482135,8
8,q02-q11,0.117137,12.849407,8
9,q02-q12_genderbalance,0.683582,16.524754,20


In [5]:
mark = alt.Chart(chi2_data).mark_point().encode(
    alt.X("p_value"),
    alt.Y("questions"),
    tooltip=["questions", "p_value", "dof", "statistic"]
)

rule = (
    alt.Chart()
    .mark_rule(color="orange")
    .encode(x=alt.datum(0.05))
)

(mark + rule).properties(
    width=500,
    height=500,
).interactive()

alt.LayerChart(...)

``p < 0.05``の項目を調べる

In [6]:
p005 = chi2_data.query("p_value < 0.05")
p005

,questions,p_value,statistic,dof
4,q02-q06,0.000047,73.110661,32
6,q02-q08,0.000002,63.665246,20
10,q02-q12_diversity,0.007419,38.625475,20
21,q02-q04_regional,0.002581,42.229956,20
24,q02-q13_binned,0.000020,121.291722,64


In [7]:
for name, hm in heatmaps.items():
    if name in list(p005.questions):
        hm.properties(
            title=name,
            width=400,
            height=400,
        ).display()

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)